In [6]:
!pip3 install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/25/92/ee1d7a00bb6b8c55755d4984fd82608603a3cc59959245068ce32e7fb808/scikit_learn-1.6.1-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/61/d8/84da3fffefb6c7d5a16968fe5b9f24c98606b165bb801bb0b8bc3985200f/scipy-1.15.2-cp311-cp311-macosx_14_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf7a134b9ab11a67b0cf0726453cedd9c5043a4fe7a35d1cefa9a1bcfb/threadpoolctl-3.5.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.

## OPTICS Algorithm Implementation

In [17]:
# Add Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import pairwise_distances
from sklearn.cluster import OPTICS as SklearnOPTICS
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

In [8]:
dataset_path = "./../datasets"

iris_dataset_path = dataset_path + "/iris.csv"                                         
ai_global_index_path = dataset_path + "/AI_index_db.csv"
global_earthquake_data_path = dataset_path + "/earthquakes.csv"

In [9]:
iris_df = pd.read_csv(iris_dataset_path)
ai_global_index_df = pd.read_csv(ai_global_index_path)
global_earthquake_data_df = pd.read_csv(global_earthquake_data_path)

datasets = {
    "iris": iris_df,
    "ai_global_index": ai_global_index_df,
    "global_earthquake": global_earthquake_data_df
}

### DBSCAN Implementation (Based on our Algorithm - see report/Part-1.pdf)

In [18]:
class OPTICSFromScratch:
    def __init__(self, eps=0.5, min_pts=5):
        self.eps = eps
        self.min_pts = min_pts
        self.reachability_ = None
        self.ordering_ = []

    def fit(self, X):
        n = len(X)
        self.reachability_ = np.full(n, np.inf)
        processed = np.zeros(n, dtype=bool)
        self.ordering_ = []

        for idx in range(n):
            if processed[idx]:
                continue

            neighbors = self._region_query(X, idx)
            processed[idx] = True
            self.ordering_.append(idx)

            if self._core_distance(X, idx, neighbors) is not None:
                seeds = []
                self._update(X, neighbors, idx, seeds, processed)

                while seeds:
                    current = seeds.pop(0)
                    if processed[current]:
                        continue
                    current_neighbors = self._region_query(X, current)
                    processed[current] = True
                    self.ordering_.append(current)

                    if self._core_distance(X, current, current_neighbors) is not None:
                        self._update(X, current_neighbors, current, seeds, processed)

    def _core_distance(self, X, idx, neighbors):
        if len(neighbors) < self.min_pts:
            return None
        dists = np.linalg.norm(X[neighbors] - X[idx], axis=1)
        return np.partition(dists, self.min_pts - 1)[self.min_pts - 1]

    def _update(self, X, neighbors, idx, seeds, processed):
        core_dist = self._core_distance(X, idx, neighbors)
        for neighbor in neighbors:
            if processed[neighbor]:
                continue
            dist = np.linalg.norm(X[neighbor] - X[idx])
            new_reach = max(core_dist, dist)
            if self.reachability_[neighbor] == np.inf:
                self.reachability_[neighbor] = new_reach
                seeds.append(neighbor)
            elif new_reach < self.reachability_[neighbor]:
                self.reachability_[neighbor] = new_reach

    def _region_query(self, X, idx):
        distances = np.linalg.norm(X - X[idx], axis=1)
        return list(np.where(distances <= self.eps)[0])


### Evaluation Function

In [ ]:
results = {}
for name, df in datasets.items():
    X = df.drop(columns=["target"]).values
    y = df["target"].values

    # Normalize the data
    X = StandardScaler().fit_transform(X)

    # Run the custom implementation
    optics = OPTICSFromScratch()
    optics.fit(X)
    custom_ordering = optics.ordering_
    print("Custom OPTICS Ordering:", optics.ordering_)
    print("Custom OPTICS Reachability Distances:", optics.reachability_)

    # Run the sklearn implementation
    skoptics = SklearnOPTICS()
    skoptics.fit(X)
    sklearn_ordering = skoptics.ordering_
    print("Sklearn OPTICS Ordering:", skoptics.ordering_)
    print("Sklearn OPTICS Reachability Distances:", skoptics.reachability_)

    # Compare the results
    results[name] = np.allclose(custom_ordering, sklearn_ordering)

# store results 
results = pd.Series(results)
results.to_csv("./../results/02-optics.csv", header=False)